In [1]:
# Install required libraries
!pip install transformers  # Installs the transformers library for NLP models
!pip install imageio       # Installs imageio to handle image files (e.g., GIFs)
!pip install gtts          # Installs Google Text-to-Speech API for text-to-speech
!pip install bert-score    # Installs the BERTScore metric library for evaluation
!pip install nltk          # Installs the NLTK library for natural language processing tasks
!pip install rouge-score   # Installs the Rouge score package for evaluation metrics

import os
import json
import random
import requests
import imageio
from PIL import Image
from io import BytesIO
import torch
from transformers import (
    BlipProcessor,
    BlipForConditionalGeneration,
    T5Tokenizer,
    T5ForConditionalGeneration,
    AdamW
)
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
# from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score as bert_score_fn
import numpy as np

# Download NLTK data
nltk.download('wordnet')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7a78130f332a1bba9ba244bc114b897e2d69c8d4e0b85ba4d8182fba589cacc2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import os
import json
import random

# Define paths
metadata_file = 'datafile.json'  # Path to your metadata file that contains GIF information
gifs_dir = 'gifs_temp'           # Directory to temporarily store GIFs

# Create temporary GIFs directory if it doesn't exist
os.makedirs(gifs_dir, exist_ok=True)

# Load metadata from the metadata file
with open(metadata_file, 'r') as f:
    metadata = json.load(f)

# Check if the dataset contains at least 300 GIFs, if not raise an error
if len(metadata) < 300:
    raise ValueError("The metadata file contains fewer than 100 GIFs.")

# Randomly select 300 GIFs from the metadata for processing
selected_gifs = random.sample(metadata, 300)

# Output the number of selected GIFs
print(f"Selected {len(selected_gifs)} random GIFs for processing.")


Selected 300 random GIFs for processing.


In [ ]:
import time

# Enhanced download function with retries
def download_gif(url, save_path, max_retries=3, backoff_factor=2):
    for attempt in range(1, max_retries + 1):
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {save_path}")
            return True
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error: {http_err} - Attempt {attempt}")
            if response.status_code == 404:
                break
        except requests.exceptions.RequestException as req_err:
            print(f"Request error: {req_err} - Attempt {attempt}")
        except Exception as e:
            print(f"Unexpected error: {e} - Attempt {attempt}")
        # Exponential backoff
        time.sleep(backoff_factor ** attempt)
    print(f"Failed to download {url}")
    return False

# Function to extract and preprocess frames from GIF
def extract_frames(gif_path, num_frames=10, frame_size=(256, 256)):
    try:
        gif = imageio.mimread(gif_path)
        total_frames = len(gif)

        if total_frames == 0:
            raise ValueError("No frames found in GIF.")

        interval = max(total_frames // num_frames, 1)
        selected_frames = [gif[i] for i in range(0, total_frames, interval)][:num_frames]

        processed_frames = []
        for frame in selected_frames:
            img = Image.fromarray(frame)
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img_resized = img.resize(frame_size)
            processed_frames.append(np.array(img_resized))

        # Pad with the last frame or zeros
        while len(processed_frames) < num_frames:
            processed_frames.append(processed_frames[-1] if processed_frames else np.zeros((frame_size[0], frame_size[1], 3), dtype=np.uint8))

        return processed_frames

    except Exception as e:
        print(f"Failed to extract frames from {gif_path}: {e}")
        return []

# Function to delete GIF after processing
def delete_gif(gif_path):
    try:
        os.remove(gif_path)
        print(f"Deleted: {gif_path}")
    except Exception as e:
        print(f"Failed to delete {gif_path}: {e}")


In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import os

# Initialize lists to store data
target_frames = []
target_texts = []

# Process each GIF
for gif in tqdm(selected_gifs, desc="Processing GIFs"):
    gif_id = gif.get('id')
    url = gif.get('url')
    reference_description = gif.get('description')

    # Validate GIF entry
    if not gif_id or not url or not reference_description:
        print(f"Invalid GIF entry: {gif}")
        continue

    # Define local path to save the GIF temporarily
    gif_filename = f"{gif_id}.gif"
    gif_path = os.path.join(gifs_dir, gif_filename)

    # Download the GIF
    success = download_gif(url, gif_path)
    if not success:
        print(f"Skipping GIF {gif_id} due to download failure.")
        continue

    # Extract frames
    selected_frames = extract_frames(gif_path, num_frames=5)

    # Validate frame extraction
    if not selected_frames:
        print(f"No frames extracted for {gif_id}. Skipping.")
        delete_gif(gif_path)
        continue

    # Append the extracted frames and corresponding text
    target_frames.append(selected_frames)
    target_texts.append(reference_description)

    # Delete the downloaded GIF
    delete_gif(gif_path)


Processing GIFs:   0%|          | 1/300 [00:00<02:46,  1.80it/s]

Downloaded: gifs_temp/gif_42570.gif
Deleted: gifs_temp/gif_42570.gif


Processing GIFs:   1%|          | 2/300 [00:00<02:17,  2.17it/s]

Downloaded: gifs_temp/gif_85128.gif
Deleted: gifs_temp/gif_85128.gif


Processing GIFs:   1%|          | 3/300 [00:01<02:10,  2.28it/s]

...

Downloaded: gifs_temp/gif_42786.gif
Deleted: gifs_temp/gif_42786.gif


Processing GIFs: 100%|█████████▉| 299/300 [02:27<00:00,  2.25it/s]

Downloaded: gifs_temp/gif_38281.gif
Deleted: gifs_temp/gif_38281.gif


Processing GIFs: 100%|██████████| 300/300 [02:28<00:00,  2.02it/s]

Downloaded: gifs_temp/gif_13864.gif
Deleted: gifs_temp/gif_13864.gif


In [ ]:
import torch
import numpy as np

# Assume target_frames is the list of GIFs, where each GIF is a list of frames (each frame is a numpy array)
processed_gifs = []

for gif in target_frames:
    # Convert list of frames (numpy arrays) into a single numpy array
    frames_array = np.array(gif)  # Convert to a single numpy array
    # Now convert the numpy array to a PyTorch tensor
    frames_tensor = torch.tensor(frames_array)  # Convert to a PyTorch tensor

    # Append the processed tensor to the list
    processed_gifs.append(frames_tensor)

# Now processed_gifs is a list of tensors, where each tensor represents a GIF


Saving the data

In [ ]:
import torch

# Move all tensors in processed_gifs to CPU
processed_gifs_cpu = [gif.cpu() for gif in processed_gifs]

# Create a dictionary to hold both lists
data = {
    'processed_gifs': processed_gifs_cpu,
    'target_texts': target_texts
}

# Save the dictionary to a file
torch.save(data, 'testdata.pth')

print("Data has been saved successfully as 'data.pth'.")


Data has been saved successfully as 'data.pth'.


Loading data when needing

In [2]:
import torch

# Load the saved data from the local file
data = torch.load('testdata.pth', map_location=torch.device('cpu'))  # Change 'cpu' to 'cuda' if loading on GPU

processed_gifs = data['processed_gifs']
target_texts = data['target_texts']

print("Data has been loaded successfully from 'data.pth'.")


<ipython-input-2-afe2d304acc8>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('testdata.pth', map_location=torch.device('cpu'))  # Change 'cpu' to 'cuda'

Data has been loaded successfully from 'data.pth'.


In [3]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration

# Define device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize processors and tokenizers
blip_processor = BlipProcessor.from_pretrained('Salesforce/blip-image-captioning-base')  # Use the same processor used during training
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')  # Use the same tokenizer used during training

# Initialize model architectures
blip_model = BlipForConditionalGeneration.from_pretrained('Salesforce/blip-image-captioning-base')  # Ensure architecture matches
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')  # Ensure architecture matches

# Load your trained state_dicts
blip_model.load_state_dict(torch.load('/content/drive/MyDrive/Models/7 epochs 1500 GIFs/blip_model.pth', map_location=device))
t5_model.load_state_dict(torch.load('/content/drive/MyDrive/Models/7 epochs 1500 GIFs/t5_model.pth', map_location=device))

# Move models to device
blip_model.to(device)
t5_model.to(device)

# Set models to evaluation mode
blip_model.eval()
t5_model.eval()

print("Trained models loaded successfully.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

<ipython-input-3-b78857e42f4b>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  blip_model.load_state_dict(torch.load('/content/drive/MyDrive/Models/7 epochs 1500 GIFs/blip

Trained models loaded successfully.


In [4]:
import torch
import numpy as np
from torch.utils.data import Dataset

class GifAggregationDataset(Dataset):
    def __init__(self, frames_list, targets, tokenizer, max_target_length=150):
        """
        frames_list: A list of lists containing frames (as tensors or numpy arrays) for each GIF.
        targets: The ground truth captions for each GIF.
        tokenizer: Tokenizer for T5 to encode the target captions.
        max_target_length: Maximum length for the target text sequences.
        """
        self.frames_list = frames_list  # List of frame sequences (e.g., list of tensors for each GIF)
        self.targets = targets  # Ground truth captions for each GIF
        self.tokenizer = tokenizer
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.frames_list)

    def __getitem__(self, idx):
        # Get frames and target for the given index
        frames = self.frames_list[idx]  # List or tensor of frames for a single GIF
        target_text = self.targets[idx]

        # Ensure frames are in the correct shape (num_frames, height, width, channels)
        if isinstance(frames, list):
            # Convert list of numpy arrays to a single numpy array, if necessary
            frames = np.array(frames)

        # If frames are numpy arrays, convert to torch tensor and make sure dimensions are correct
        if isinstance(frames, np.ndarray):
            frames = torch.tensor(frames, dtype=torch.float32)  # Convert to float32 tensor for PyTorch

        # Reshape or permute frames to ensure the shape is (num_frames, height, width, channels)
        if frames.ndim == 4 and frames.shape[-1] == 3:
            # Shape is (num_frames, height, width, channels), which is correct for PIL conversion
            pass
        elif frames.ndim == 4 and frames.shape[1] == 3:
            # If frames have shape (num_frames, channels, height, width), change to (num_frames, height, width, channels)
            frames = frames.permute(0, 2, 3, 1)  # (N, C, H, W) -> (N, H, W, C)

        # Tokenize the target caption (ground-truth description)
        target_encoding = self.tokenizer(
            target_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_target_length,
            return_tensors="pt"
        )

        # Replace padding token ID's of the labels by -100 to ignore them in the loss
        labels = target_encoding.input_ids.squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'frames': frames,  # Provide frames directly in tensor format (num_frames, height, width, channels)
            'labels': labels,  # Provide tokenized ground truth captions for T5
            'reference_description': target_text
        }


In [5]:
# Define the Test Dataset
test_dataset = GifAggregationDataset(
    frames_list=processed_gifs,    # List of tensors representing frames for each test GIF
    targets=target_texts,          # Ground truth aggregated captions for test GIFs
    tokenizer=t5_tokenizer,             # Tokenizer for processing target texts
    max_target_length=150                # Max length for target captions
)

# Create the Test DataLoader
test_dataloader = DataLoader(
    test_dataset,
    batch_size=16,          # Adjust based on GPU memory; 16 is consistent with your train/val
    shuffle=False,          # No need to shuffle test data
    num_workers=0,          # Set >0 if using multiple CPU cores for faster data loading
    pin_memory=True         # Speeds up data transfer to GPU
)


In [6]:
import torch
from tqdm import tqdm
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
import bert_score
from PIL import Image

def evaluate_model(blip_model, t5_model, blip_processor, tokenizer, dataloader, device='cuda' if torch.cuda.is_available() else 'cpu'):
    blip_model.eval()  # Set BLIP to evaluation mode
    t5_model.eval()    # Set T5 to evaluation mode

    # Metrics containers
    all_references = []
    all_candidates = []
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    # meteor_scores_list = []

    # ROUGE scorer initialization
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # BERTScore
    bert_score_fn = bert_score.score

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Step 1: Prepare frames for BLIP
            frames = batch['frames']  # Assuming shape [batch_size, num_frames, C, H, W] or [batch_size, num_frames, H, W, C]
            reference_descriptions = batch['reference_description']  # List of reference captions

            batch_size, num_frames = frames.shape[0], frames.shape[1]

            frame_captions = []

            for i in range(batch_size):
                single_captions = []
                for j in range(num_frames):
                    frame = frames[i, j]  # Shape depends on data format

                    # Determine frame shape and permute if necessary
                    if frame.ndim == 3:
                        # Possible shapes:
                        # [C, H, W] or [H, W, C]
                        if frame.shape[0] == 3:
                            # [C, H, W] -> [H, W, C]
                            frame = frame.permute(1, 2, 0)
                        # Else assume [H, W, C], no change needed
                    else:
                        raise ValueError(f"Unexpected frame shape: {frame.shape}")

                    # Convert to CPU and numpy
                    frame_np = frame.cpu().numpy().astype('uint8')  # Ensure dtype is uint8

                    # Convert to PIL Image
                    try:
                        img = Image.fromarray(frame_np).convert('RGB')
                    except Exception as e:
                        print(f"Error converting frame to image: {e}")
                        continue  # Skip this frame

                    # Generate caption using BLIP
                    inputs = blip_processor(img, return_tensors="pt").to(device)
                    outputs = blip_model.generate(**inputs, max_length=50)  # Limit caption length
                    caption = blip_processor.decode(outputs[0], skip_special_tokens=True)
                    single_captions.append(caption)

                # Concatenate captions for the current example
                concatenated_caption = " ".join(single_captions)
                frame_captions.append(concatenated_caption)

            # Step 2: Tokenize the concatenated captions as a batch
            max_length = 128  # Adjust based on GPU capacity and desired sequence length
            t5_inputs = tokenizer(
                frame_captions,  # List of concatenated captions, one per example in the batch
                max_length=max_length,
                padding='max_length',
                truncation=True,
                return_tensors="pt"
            ).to(device)

            # Step 3: Generate predictions using T5
            outputs = t5_model.generate(
                input_ids=t5_inputs.input_ids,
                attention_mask=t5_inputs.attention_mask,
                max_length=150,
                num_beams=4,
                early_stopping=True
            )

            # Step 4: Decode predictions
            predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)

            # Step 5: Evaluate the predictions
            for ref, pred in zip(reference_descriptions, predictions):
                ref_tokens = ref.split()
                pred_tokens = pred.split()

                # Append references and predictions for BLEU
                all_references.append([ref_tokens])
                all_candidates.append(pred_tokens)

                # ROUGE scores calculation
                scores = scorer.score(ref, pred)
                rouge1_scores.append(scores['rouge1'].fmeasure)
                rouge2_scores.append(scores['rouge2'].fmeasure)
                rougeL_scores.append(scores['rougeL'].fmeasure)

                # METEOR score calculation
                # meteor = meteor_score([ref_tokens], pred_tokens)
                # meteor_scores_list.append(meteor)

    # Calculate BLEU score
    bleu = corpus_bleu(all_references, all_candidates)

    # Calculate average ROUGE scores
    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
    avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

    # Calculate average METEOR score
    # avg_meteor = sum(meteor_scores_list) / len(meteor_scores_list) if meteor_scores_list else 0

    # Calculate BERT Score
    P, R, F1 = bert_score_fn(
        [' '.join(cand) for cand in all_candidates],
        [' '.join(ref[0]) for ref in all_references],
        lang='en',
        verbose=True)
    avg_bert_f1 = F1.mean().item()

    # Compile metrics
    metrics = {
        'BLEU': bleu,
        'ROUGE-1': avg_rouge1,
        'ROUGE-2': avg_rouge2,
        'ROUGE-L': avg_rougeL,
        # 'METEOR': avg_meteor,
        'BERT_F1': avg_bert_f1
    }

    return metrics


In [ ]:
# After training each epoch, evaluate the model on the validation set
metrics = evaluate_model(blip_model, t5_model, blip_processor, t5_tokenizer, test_dataloader, device)
print(f"Test Metrics Model V1: {metrics}")


Evaluating: 100%|██████████| 19/19 [05:35<00:00, 17.64s/it]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/10 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 2.36 seconds, 126.96 sentences/sec
Test Metrics Model V1: {'BLEU': 0.06812399819740517, 'ROUGE-1': 0.36283740952837173, 'ROUGE-2': 0.1208941130679991, 'ROUGE-L': 0.32716171295477103, 'BERT_F1': 0.9016732573509216}


In [ ]:
# After training each epoch, evaluate the model on the validation set
metrics = evaluate_model(blip_model, t5_model, blip_processor, t5_tokenizer, test_dataloader, device)
print(f"Test Metrics Model V2: {metrics}")

Evaluating: 100%|██████████| 19/19 [05:37<00:00, 17.74s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/10 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 1.67 seconds, 179.13 sentences/sec
Test Metrics Model V2: {'BLEU': 0.06812399819740517, 'ROUGE-1': 0.36283740952837173, 'ROUGE-2': 0.1208941130679991, 'ROUGE-L': 0.32716171295477103, 'BERT_F1': 0.9016732573509216}


In [8]:
# After training each epoch, evaluate the model on the validation set
metrics = evaluate_model(blip_model, t5_model, blip_processor, t5_tokenizer, test_dataloader, device)
print(f"Test Metrics Model V3: {metrics}")

Evaluating: 100%|██████████| 19/19 [05:42<00:00, 18.02s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/9 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 1.51 seconds, 198.45 sentences/sec
Test Metrics Model V3: {'BLEU': 0.06573281519797997, 'ROUGE-1': 0.3650295021009056, 'ROUGE-2': 0.12514736381462965, 'ROUGE-L': 0.33186308303216555, 'BERT_F1': 0.9034396409988403}
